In [2]:
$:.unshift File.expand_path("..")

require 'market.rb'
require 'script/helpers.rb'
require 'daru/view'

Daru::View.plotting_library = :highcharts

# Rewriting and monkeypatching to remove the NMatrix requirement from nyaplot
# Daru makes its own NMatrix, but it doesn't have #to_nm.
class Array
  def mean
    ary = self.to_a.compact
    ary.inject(0) {|s, v| s + v } / ary.size.to_f
  end
end

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

"\n   /* BEGIN highstock.js */\n\n/*\n Highstock JS v6.1.1 (2018-06-27)\n\n (c) 2009-2016 Torstein Honsi\n\n License: www.highcharts.com/license\n*/\n(function(W,L){\"object\"===typeof module&&module.exports?module.exports=W.document?L(W):L:W.Highcharts=L(W)})(\"undefined\"!==typeof window?window:this,function(W){var L=function(){var a=\"undefined\"===typeof W?window:W,C=a.document,D=a.navigator&&a.navigator.userAgent||\"\",E=C&&C.createElementNS&&!!C.createElementNS(\"http://www.w3.org/2000/svg\",\"svg\").createSVGRect,q=/(edge|msie|trident)/i.test(D)&&!a.opera,n=-1!==D.indexOf(\"Firefox\"),f=-1!==D.indexOf(\"Chrome\"),r=n&&4>parseInt(D.split(\"Firefox/\")[1],\n10);return a.Highcharts?a.Highcharts.error(16,!0):{product:\"Highstock\",version:\"6.1.1\",deg2rad:2*Math.PI/360,doc:C,hasBidiBug:r,hasTouch:C&&void 0!==C.documentElement.ontouchstart,isMS:q,isWebKit:-1!==D.indexOf(\"AppleWebKit\"),isFirefox:n,isChrome:f,isSafari:!f&&-1!==D.indexOf(\"Safari\"),isTouchDevice:/(Mobile|Android|Win

:mean

# Gather the results

We're just going to take all of the cached results from 2008-2020. We'll have access to the buy bar, sell bar (if any), hold time, and ROI (-1 if the bar is unsold).

In [3]:
res = (2008..2020).map {|year| simulate(:year => year).results }.flatten
res.size

1233

# Cycle through each 365-day period

Cycle through each 365-day period and calculate the ROI of the buy signals that occurred in that period.

In [4]:
def cycle(data, start: nil, finish: nil, days: 365)
  res = []
  while start <= finish
    fin = start + days.days
    subset = data.filter {|h| h[:buy].date > start && h[:buy].date < fin }
    res << [start.to_i * 1000, subset.empty? ? 0 : subset.map {|r| r[:ROI] }.mean]
    
    start += 1.day
  end
  
  res
end

:cycle

In [5]:
start = Time.parse('1 jan 2008')
finish = Time.parse('1 jan 2020')

data_365 = cycle res, :days => 365, :start => start, :finish => finish
data_90  = cycle res, :days => 90, :start => start, :finish => finish
data_30  = cycle res, :days => 30, :start => start, :finish => finish

nil

In [14]:
opts = {:xAxis => {:labels => {:formatter => "function() {
                                                return Highcharts.dateFormat('%b/%e/%Y', this.value);
                                              }".js_code},
                   :type => "datetime"},
        :yAxis => [{:title => {:text => "1-year ROI"}, :min => -1},
                   {:title => {:text => "X-day ROI"}, :opposite => 'true'}],
        :legend => {:layout => 'vertical',
                    :align => 'right',
                    :verticalAlign => 'middle'},
        :tooltip => {:xDateFormat => "%Y-%m-%d"}}

plt = Daru::View::Plot.new
plt.chart.options = opts
plt.chart.series_data.shift # pop the first empty series off
plt.add_series :data => data_365, :type => :line, :name => '1-year ROI'

# Commented-out because it distorts the graph in a way that makes it hard to read.
plt.add_series :data => data_90, :type => :line, :name => '90-day ROI', :yAxis => 1
plt.add_series :data => data_30, :type => :line, :name => '30-day ROI', :yAxis => 1

plt.show_in_iruby

"\n\nvar event = document.createEvent(\"HTMLEvents\");\nevent.initEvent(\"load_highcharts\", false, false);\nwindow.dispatchEvent(event);\nconsole.log(\"Finish loading highchartsjs\");\n"

"              <script type=\"text/javascript\">\n        $(function() {\n\n                var options = { \"xAxis\": { \"labels\": { \"formatter\": function() {\n                                                return Highcharts.dateFormat('%b/%e/%Y', this.value);\n                                              } },\"type\": \"datetime\" },\"yAxis\": [ { \"title\": { \"text\": \"1-year ROI\" },\"min\": -1 },{ \"title\": { \"text\": \"X-day ROI\" },\"opposite\": \"true\" } ],\"legend\": { \"layout\": \"vertical\",\"align\": \"right\",\"verticalAlign\": \"middle\" },\"tooltip\": { \"xDateFormat\": \"%Y-%m-%d\" },\"chart\": { \"renderTo\": \"nadfhmzxobl\" },\"series\": [{ \"data\": [ [ 1199163600000,1.533156578867726 ],[ 1199250000000,1.5303150877393896 ],[ 1199336400000,1.5303150877393896 ],[ 1199422800000,1.5346306521338768 ],[ 1199509200000,1.5346306521338768 ],[ 1199595600000,1.5346306521338768 ],[ 1199682000000,1.5390150256215591 ],[ 1199768400000,1.5390150256215591 ],[ 1199854800000,1.5433455285687478 ],[ 1199941200000,1.5433455285687478 ],[ 1200027600000,1.5433455285687478 ],[ 1200114000000,1.5433455285687478 ],[ 1200200400000,1.5433455285687478 ],[ 1200286800000,1.5433455285687478 ],[ 1200373200000,1.5305090386414306 ],[ 1200459600000,1.5305090386414306 ],[ 1200546000000,1.5363269799464374 ],[ 1200632400000,1.542350531184079 ],[ 1200718800000,1.542350531184079 ],[ 1200805200000,1.542350531184079 ],[ 1200891600000,1.542350531184079 ],[ 1200978000000,1.542235337435786 ],[ 1201064400000,1.5387384546957745 ],[ 1201150800000,1.5396332994748172 ],[ 1201237200000,1.540876077546404 ],[ 1201323600000,1.540876077546404 ],[ 1201410000000,1.540876077546404 ],[ 1201496400000,1.543088659732325 ],[ 1201582800000,1.5421336266334946 ],[ 1201669200000,1.5394048809307657 ],[ 1201755600000,1.5394048809307657 ],[ 1201842000000,1.5422998012482836 ],[ 1201928400000,1.5422998012482836 ],[ 1202014800000,1.5422998012482836 ],[ 1202101200000,1.5377403231005813 ],[ 1202187600000,1.5408410623397963 ],[ 1202274000000,1.5398238239650701 ],[ 1202360400000,1.5464311933944346 ],[ 1202446800000,1.5485256157380203 ],[ 1202533200000,1.5485256157380203 ],[ 1202619600000,1.5485256157380203 ],[ 1202706000000,1.5485256157380203 ],[ 1202792400000,1.5485256157380203 ],[ 1202878800000,1.5473881027435217 ],[ 1202965200000,1.5570301994371631 ],[ 1203051600000,1.566328550129689 ],[ 1203138000000,1.566328550129689 ],[ 1203224400000,1.566328550129689 ],[ 1203310800000,1.566328550129689 ],[ 1203397200000,1.5744515440787363 ],[ 1203483600000,1.5841650598493622 ],[ 1203570000000,1.5951786613326964 ],[ 1203656400000,1.6110914045433293 ],[ 1203742800000,1.6110914045433293 ],[ 1203829200000,1.6110914045433293 ],[ 1203915600000,1.6297030882130714 ],[ 1204002000000,1.6408823080831634 ],[ 1204088400000,1.6470932840926789 ],[ 1204174800000,1.6500690982890782 ],[ 1204261200000,1.6541543113598247 ],[ 1204347600000,1.6541543113598247 ],[ 1204434000000,1.6541543113598247 ],[ 1204520400000,1.6697410595726527 ],[ 1204606800000,1.7032773646139523 ],[ 1204693200000,1.731003153621533 ],[ 1204779600000,1.7441545678777506 ],[ 1204866000000,1.7927448786921916 ],[ 1204952400000,1.7927448786921916 ],[ 1205038800000,1.8421227612964368 ],[ 1205125200000,1.8675555371388008 ],[ 1205211600000,1.895467406848632 ],[ 1205298000000,1.9014194054768 ],[ 1205384400000,1.9014194054768 ],[ 1205470800000,1.9014194054768 ],[ 1205557200000,1.9014194054768 ],[ 1205643600000,1.9071039793844367 ],[ 1205730000000,1.9142749626271 ],[ 1205816400000,1.9309488816346365 ],[ 1205902800000,1.93503753164544 ],[ 1205989200000,1.9470053685641182 ],[ 1206075600000,1.9470053685641182 ],[ 1206162000000,1.9470053685641182 ],[ 1206248400000,1.954877661404275 ],[ 1206334800000,1.954877661404275 ],[ 1206421200000,1.956837662330186 ],[ 1206507600000,1.95913119295456 ],[ 1206594000000,1.9601755096102422 ],[ 1206680400000,1.9601755096102422 ],[ 1206766800000,1.9601755096102422 ],[ 1206853200000,1.9577250886475699 ],[ 1206939600000,1.

# Why are there dips every ~2.7 years?

What does this graph tell us? I have literally no idea.